# Manual arrival time picking

This notebook gives an example to manually pick arrival times for DAS data. The picking happens interactively, we plot the data and by double-clicking on the figure, we place a pick. We can zoom the figure in and out to focus on details in the data. When a few picks are selected, we close the figure and linearly interpolate between the picks to ensure arrival times along that entire fibre section. 

This notebook and the data example accompany the publication "Submarine fibre-optic sensing potential for regional seismicity monitoring near Santorini and Kolumbo Volcano" (2025).

“Copyright Lars Gebraad, Seismology and Wave Physics, ETH Zürich.”

In [ ]:
# import the imports

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backend_bases import MouseButton
from scipy.ndimage import gaussian_filter
from scipy import interpolate
from datetime import datetime, timedelta
import pandas as pd

# IMPORTANT: we need to get the figure as a pop-up to ensure we can interact with it
%matplotlib qt 

## Parameters

In [30]:
# data parameters

t0 = datetime(2021, 10, 21, 22, 23, 37, 677363) 
t1 = datetime(2021, 10, 21, 22, 23, 43, 677363)
d0 = 0
d1 = 34000

current_filter = 1

## Functions

These are the functions to plot the data and to be able to interact with the figure with mouse clicks and key presses.

In [3]:
def plot(signal, t1, t2, fs=40, d1=0, d2=45, picks=None, s=None, start_window=None, end_window=None, save=False, shift_fit=None, dpi=150, cbar=False, clip=0):
    """
    plotting function
    """

    if clip == 0:
        clip = np.max(np.abs(signal))
        
    fig, ax = plt.subplots(dpi=dpi)
    
    dist = [d1, d2]
    
    plt.xlabel('Time (s)')
    plt.ylabel('Distance along cable (km)')
    
    if clip != None:
        im = plt.imshow(signal, cmap='seismic', vmin=-clip, vmax=clip, origin='lower', aspect='auto', extent=(0, (t2-t1).seconds, d1, d2))
    else:
        im = plt.imshow(signal, cmap='seismic', origin='lower', aspect='auto', extent=(0, (t2-t1).seconds, d1, d2))
   
    
    if cbar:
        plt.colorbar(label='nanostrain rate')
    

    return fig, ax, dist, im

In [4]:
def find_nearest(array, value):
    """Utility function that finds the argument of the entry with the nearest
    value to a number.
    """
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

In [5]:
def onclick(event):
    """Event handler for double left click and double right click"""

    # Double right
    if event.dblclick and event.button is MouseButton.RIGHT:
        # Remove 10 nearest picks
        index_of_pick = find_nearest(channel_dist, event.ydata)
        picks[index_of_pick - 5 : index_of_pick + 5] = 0.0

    # Double left
    if event.dblclick and event.button is MouseButton.LEFT:
        # Add pick on selected channel
        index_of_pick = find_nearest(channel_dist, event.ydata)
        picks[index_of_pick] = event.xdata

    # In either event we need to update the plot, but not on single clicks
    if event.dblclick:
        # Update plots of picks after either double click
        line[0].set_xdata(picks[picks > 0])
        line[0].set_ydata(channel_dist[picks > 0])
        scat.set_offsets(np.vstack((picks, channel_dist)).T)
        fig.canvas.draw()
        fig.canvas.flush_events()

In [6]:
def on_press(event):
    """Event handler for key presses"""

    global current_filter

    # Pressed += key for more smoothing.
    if event.key == "=":
        current_filter += 1
        print(f"Current_filter: {current_filter}")

    # Pressed _- key for less smoothing.
    if event.key == "-":
        current_filter -= 1
        current_filter = max(current_filter, 0)
        print(f"Current_filter: {current_filter}")

    # In either case, update plot of data
    if event.key == "-" or event.key == "=":
        dataf = gaussian_filter(data, sigma=current_filter)
        image.set_data(dataf)
        fig.canvas.draw()
        fig.canvas.flush_events()

## open the data

This data is already preprocessed with tapering, detrending, demeaning and a bandpass filter between 2 and 19 Hz with a sampling rate of 40 Hz. The data occurs between t0 and t1 in time, and d0 and d1 along the fibre-optic cable.

In [12]:
data = np.load('kolumbo_earthquake.npy')

channel_dist = np.linspace(d0, d1, data.shape[0])/1000

In [13]:
picks = np.zeros(data.shape[0])

## Picking !

This cell will create the figure that we use to pick the data. A Gaussian filter can be applied to the data during the picking, which may help visualise the first arrivals. The Gaussian filter is controlled through the current_filter parameter, which corresponds to the standard deviation for the Gaussian kernel.

We have the following options to interact with the figure:
- **double left click**: place a pick at that data point
- **double right click**: remove the pick at that data point
- **=**: increase the current_filter parameter by 1
- **-**: decrease the current_filter parameter by 1



In [32]:
fig, ax, dist, image = plot(data, t0, t1, fs=40, d1=d0/1000, d2=d1/1000, cbar=True, clip=200)

# Attach both handlers to the figure we just made
fig.canvas.mpl_connect("key_press_event", on_press)
fig.canvas.mpl_connect("button_press_event", onclick)

xlim = ax.get_xlim()

# Plot current picks
scat = plt.scatter(picks, channel_dist, s=1, c="g")

# Plot interpolation of picks
line = plt.plot(
    picks[picks > 0], channel_dist[picks > 0], linestyle="dotted", c="k"
)
line[0].set_linewidth(0.5)

plt.show()

## interpolate

Interpolate linearly between the picked arrivals.

In [33]:
# interpolation of picks

x0 = channel_dist[picks>0]
f = interpolate.interp1d(x0, picks[picks>0])
id1 = np.where(channel_dist == x0[0])[0][0]
id2 = np.where(channel_dist == x0[-1])[0][0]

p = f(channel_dist[id1:id2])

## Save

Store the results.

In [34]:
# convert relative picks to absolute time

abs_t = []
for ii in range(len(p)):
    abs_t.append(t0 + timedelta(seconds=p[ii]))

In [36]:
# save results in a csv file

dict = {'distance along fibre (km)': channel_dist[id1:id2], 'relative arrival time (s)':p-p.min(), 'absolute arrival time (UTC)':abs_t}
df = pd.DataFrame(dict)
df.to_csv('manual_picks.csv')